In [40]:
from datasets import load_dataset
from io import BytesIO
import torch
import torchaudio
from spauq.core.metrics import spauq_eval

In [2]:
dataset = load_dataset("danjacobellis/aria_ea_audio_preprocessed",split='validation').with_format("torch")

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

In [3]:
sample = dataset[0]
audio = sample['audio'].permute(1,0)

In [51]:
def mp3_compress(audio,fs,qscale=9):
    config = torchaudio.io.CodecConfig(qscale=qscale)
    size_bytes = []
    recovered = []
    for channel in audio:
        with BytesIO() as f:
            torchaudio.save(f, channel.unsqueeze(0), format="mp3", sample_rate=fs, compression=config)
            size_bytes.append(len(f.getvalue()))
            f.seek(0)
            recovered.append(torchaudio.load(f,format="mp3")[0])
    recovered = torch.cat(recovered)
    return recovered, size_bytes

In [52]:
rec, size = mp3_compress(audio,48000)

In [55]:
eval_output = spauq_eval(reference=audio,estimate=rec,fs=48000)

/home/dgj335/.local/lib/python3.10/site-packages/spauq/core/preprocessing.py:325: UserWarning: No forgive_mode specified, defaulting to `none`
  warnings.warn(


In [56]:
eval_output

{'SSR': 20.46714262671256, 'SRR': 9.419561357341836}